In [1]:
import numpy as np

# Задание 1

### Навигационный эллипсоид ПЗ-90:

In [2]:
a = 6378136

In [3]:
e2 = 0.00669436619

### Гринвичские координаты в задании:

In [4]:
nu1 = 2846242.89

In [5]:
nu2 = 2198707.07

In [6]:
nu3 = 5250015.07

### Вычисление географических координат:

In [7]:
angle_lambda = np.arctan(nu2 / nu1)

In [8]:
phi0 = np.arctan(nu3 / np.sqrt(nu1**2 + nu2**2))

In [9]:
Phi = phi0 + 0.5 * e2 * np.sin(2 * phi0)

In [10]:
H = np.sqrt(nu1**2 + nu2**2 + nu3**2) - a + 0.5 * a * e2 * np.sin(Phi)**2

In [11]:
while True:
    sin_phi = np.sin(Phi)
    cos_phi = np.cos(Phi)

    Re = a / np.sqrt(1 - e2 * sin_phi**2)
    dRe_dPhi = - a * e2 * sin_phi * cos_phi / (1 - e2 * sin_phi**2)**(3/2)
    dF1_dPhi = - dRe_dPhi * cos_phi + (Re + H) * sin_phi
    dF2_dPhi = - (1 - e2) * dRe_dPhi * sin_phi - ((1 - e2) * Re + H) * cos_phi
    dF1_dH = - cos_phi
    dF2_dH = - sin_phi
    J = np.array([[dF2_dH, - dF1_dH], [- dF2_dPhi, dF1_dPhi]])
    F1 = np.sqrt(nu1**2 + nu2**2) - (Re + H) * cos_phi
    F2 = nu3 - ((1 - e2) * Re + H) * sin_phi
    F = np.array([[F1], [F2]])
    delta = dF1_dPhi * dF2_dH - dF2_dPhi * dF1_dH
    Phi_H_cur = np.array([[Phi], [H]])
    Phi_H_next = Phi_H_cur - np.matmul(J, F) / delta
    
    dPhi = Phi_H_next[0][0] - Phi
    dH = Phi_H_next[1][0] - H
    if dPhi**2 * a**2 + dH**2 <= 0.001**2:
        geo_coords = (Phi_H_next[0][0], Phi_H_next[1][0])
        break 
    Phi, H = Phi_H_next[0][0], Phi_H_next[1][0]

In [12]:
geo_coords  

(0.9732944911091262, 250.96802480283876)

In [13]:
np.pi

3.141592653589793

In [14]:
angle_lambda*180./np.pi

37.68589002760919

In [15]:
print(f"lambda = {(angle_lambda*180./np.pi)//1:.0f}◦ {(angle_lambda*180./np.pi)%1*60//1:.0f}' {(angle_lambda*180./np.pi)%1*60%1*60:.4f}''")

lambda = 37◦ 41' 9.2041''


In [16]:
print(f"phi = {(geo_coords[0]*180./np.pi)//1:.0f}◦ {(geo_coords[0]*180./np.pi)%1*60//1:.0f}' {(geo_coords[0]*180./np.pi)%1*60%1*60:.4f}''")

phi = 55◦ 45' 56.3996''


In [17]:
print(f"h = {geo_coords[1]:.4f}")

h = 250.9680


### Вычисление геоцентрических координат:

In [18]:
R = np.sqrt(nu1**2 + nu2**2 + nu3**2)

In [19]:
Phi01 = np.arcsin(nu3 / R)

In [20]:
R_earth = a * np.sqrt((1-e2) / (1 - e2 * np.cos(Phi01)**2))

In [21]:
H0 = R - R_earth

In [22]:
print(f"lambda = {(angle_lambda*180./np.pi)//1:.0f}◦ {(angle_lambda*180./np.pi)%1*60//1:.0f}' {(angle_lambda*180./np.pi)%1*60%1*60:.4f}''")

lambda = 37◦ 41' 9.2041''


In [23]:
print(f"phi = {(Phi01*180./np.pi)//1:.0f}◦ {(Phi01*180./np.pi)%1*60//1:.0f}' {(Phi01*180./np.pi)%1*60%1*60:.4f}''")

phi = 55◦ 35' 11.2476''


In [24]:
print(f"h0 = {H0:.4f}")

h0 = 250.9693


# Задание 2

lambda = 5◦40'9.6784''

phi = 7◦3'9.1562''

h = 652.713

In [25]:
lambda2 = (5 + 40/60 + 9.6784/3600)*np.pi/180.
lambda2

0.09894891315365785

In [26]:
phi2 = (7 + 3/60 + 9.1562/3600)*np.pi/180.
phi2

0.12309010277586997

In [27]:
h2 = 652.713

### Гринвичские координаты

In [28]:
Re2 = a / np.sqrt(1 - e2 * np.sin(phi2)**2)
Re2

6378457.853265435

In [29]:
nu1_2 = (Re2+h2)*np.cos(phi2)*np.cos(lambda2)
nu1_2

6299879.024121981

In [30]:
nu2_2 = (Re2+h2)*np.cos(phi2)*np.sin(lambda2)
nu2_2

625408.6177478718

In [31]:
nu3_2 = (Re2+h2)*np.sin(phi2) - Re2*e2*np.sin(phi2)
nu3_2

777981.4273182291

### Модель ГЛОНАСС удельной силы тяготения в гринвичских осях 

In [32]:
mu = 398600.44 * 10**9

In [33]:
C20 = -1082.6257 * 10**(-6)

In [34]:
u = 7.2921157 * 10**(-5)

In [35]:
r = np.sqrt(nu1_2**2 + nu2_2**2 + nu3_2**2)

In [36]:
g_0_nu1 = -mu/r**3 * (1 + 3/2 * C20 * a**2 / r**2 * (5 * nu3_2**2 / r**2 - 1))*nu1_2

In [37]:
g_0_nu2 = -mu/r**3 * (1 + 3/2 * C20 * a**2 / r**2 * (5 * nu3_2**2 / r**2 - 1))*nu2_2

In [38]:
g_0_nu3 = -mu/r**3 * (1 + 3/2 * C20 * a**2 / r**2 * (5 * nu3_2**2 / r**2 - 3))*nu3_2

In [39]:
g_0_nu = np.array([[g_0_nu1], [g_0_nu2], [g_0_nu3]])

Высотная поправка в географических осях:

In [40]:
h2_corr_geo = np.array([[0], [u*np.cos(phi2)], [u*np.sin(phi2)]])

In [41]:
A_nu_x0 = np.array([[-np.sin(lambda2), -np.cos(lambda2)*np.sin(phi2), np.cos(lambda2)*np.cos(phi2)], 
                    [np.cos(lambda2), -np.sin(lambda2)*np.sin(phi2), np.sin(lambda2)*np.cos(phi2)], 
                    [0, np.cos(phi2), np.sin(phi2)]])

Поправка в гринвичских осях

In [42]:
h2_corr_gr = np.dot(A_nu_x0, h2_corr_geo)

##### Сила тяжести модели ГЛОНАСС в гринвичских осях

In [43]:
g_nu = g_0_nu + h2_corr_gr
g_nu

array([[-9.69109578],
       [-0.96206527],
       [-1.20057562]])

### Формула Гельмерта:

In [44]:
g_Gel = 9.78030*(1+0.005302*(np.sin(phi2))**2 - 0.000007*(np.sin(2*phi2))**2) - 0.00014 - 2*1.543*10**(-6)*h2
g_Gel

9.778923368164968

Вектор удельной силы тяготения в географических осях

In [51]:
g_0_geo = np.array([[0], 
                    [u**2 * (Re2+h2)*np.cos(phi2)*np.sin(phi2)], 
                    [-g_Gel - u**2 * (Re2+h2)*np.cos(phi2)**2]])
g_0_geo

array([[ 0.00000000e+00],
       [ 4.13327932e-03],
       [-9.81233291e+00]])

Сила тяжести в географических осях

In [46]:
g_geo = g_0_geo + h2_corr_geo

##### Сила тяжести модели Гельмерта в гринвичских осях:

In [47]:
g_geo_nu = np.dot(A_nu_x0, g_geo)
g_geo_nu

array([[-9.69096413],
       [-0.9620522 ],
       [-1.20057851]])

Норма модели ГЛОНАСС:

In [48]:
g_nu_norm = np.sqrt(g_nu[0]**2 + g_nu[1]**2 + g_nu[1]**2)
g_nu_norm

array([9.78613696])

Норма модели Гельмерта:

In [49]:
g_geo_nu_norm = np.sqrt(g_geo_nu[0]**2 + g_geo_nu[1]**2 + g_geo_nu[1]**2)
g_geo_nu_norm

array([9.78600402])

(модель ГЛОНАСС) - (модель Гельмерта):

In [50]:
g_nu_norm - g_geo_nu_norm

array([0.00013293])